# Quantile crossing

In [ ]:
import pickle
import os

import cmocean
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import quantnn.quantiles as qq
from scipy import stats
from sklearn.isotonic import IsotonicRegression
import xarray as xr

plt.style.use('ccic.files.ccic')

def isotonic_regression(quantile_levels, predicted_quantiles):
    """Returns isotonic quantiles"""
    return IsotonicRegression(increasing=True, out_of_bounds='clip').fit(quantile_levels, predicted_quantiles).predict(quantile_levels)

In [ ]:
# Load dataset
if False:
    ds = xr.open_dataset(os.path.join(os.environ['ANALYSES'], 'quantile_crossing/GridSat_training_data_June06.nc')).load()
    ds['tiwp_distribution_isotonic'] = (ds.tiwp_distribution.dims, np.apply_along_axis(lambda x: isotonic_regression(ds.quantiles_tiwp.values, x), 1, ds.tiwp_distribution.values))
    ds['tiwp_fpavg_distribution_isotonic'] = (ds.tiwp_fpavg_distribution.dims, np.apply_along_axis(lambda x: isotonic_regression(ds.quantiles_tiwp.values, x), 1, ds.tiwp_fpavg_distribution.values))
    ds['tiwc_distribution_isotonic'] = (ds.tiwc_distribution.dims, np.apply_along_axis(lambda x: isotonic_regression(ds.quantiles_tiwc.values, x), 2, ds.tiwc_distribution.values))
    ds.to_netcdf(os.path.join(os.environ['ANALYSES'], 'quantile_crossing/GridSat_training_data_June06_with_isotonic_regression.nc'))
else:
    ds = xr.open_dataset(os.path.join(os.environ['ANALYSES'], 'quantile_crossing/GridSat_training_data_June06_with_isotonic_regression.nc')).load()

In [ ]:
# Compute spearman for tiwp and tiwc

f_spearman_tiwp = lambda x: stats.spearmanr(x, ds.quantiles_tiwp, nan_policy='raise', alternative='two-sided')
f_spearman_tiwc = lambda x: stats.spearmanr(x, ds.quantiles_tiwc, nan_policy='raise', alternative='two-sided')

# If-else to compute or read from disk

if False:
    spearman_tiwp = np.apply_along_axis(f_spearman_tiwp, 1, ds.tiwp_distribution)
    with open('spearman_tiwp.pickle', 'wb') as handle:
        pickle.dump(spearman_tiwp, handle)
else:
    with open('spearman_tiwp.pickle', 'rb') as handle:
        spearman_tiwp = pickle.load(handle)

if False:
    spearman_tiwp_fpavg = np.apply_along_axis(f_spearman_tiwp, 1, ds.tiwp_fpavg_distribution)
    with open('spearman_tiwp_fpavg.pickle', 'wb') as handle:
        pickle.dump(spearman_tiwp_fpavg, handle)
else:
    with open('spearman_tiwp_fpavg.pickle', 'rb') as handle:
        spearman_tiwp_fpavg = pickle.load(handle)

if False:
    spearman_tiwc = np.apply_along_axis(f_spearman_tiwc, 2, ds.tiwc_distribution)
    with open('spearman_tiwc.pickle', 'wb') as handle:
        pickle.dump(spearman_tiwc, handle)
else:
    with open('spearman_tiwc.pickle', 'rb') as handle:
        spearman_tiwc = pickle.load(handle)

In [ ]:
print("Minimum $rho_S$")
print("  tiwp      :", spearman_tiwp[:,0].min())
print("  tiwp_fpavg:", spearman_tiwp_fpavg[:,0].min())
print("  tiwc      :", spearman_tiwc[:,:,0].min())
print("Maximum p-value")
print("  tiwp      :", spearman_tiwp[:,1].max())
print("  tiwp_fpavg:", spearman_tiwp_fpavg[:,1].max())
print("  tiwc      :", spearman_tiwc[:,:,1].max())

In [ ]:
print("Median rho_S for")
print("  tiwp      :", np.median(spearman_tiwp[:,0].flatten()))
print("  tiwp_fpavg:", np.median(spearman_tiwp_fpavg[:,0].flatten()))
print("  tiwc      :", np.median(spearman_tiwc[:,:,0].flatten()))

In [ ]:
print("Minimum $rho_S$ for TIWC per level")
for i in range(20):
    print(f"Level {i:02d}", spearman_tiwc[:,i,0].min())

In [ ]:
print("Maximum p-value for TIWC per level")
for i in range(20):
    print(f"Level {i:02d}", spearman_tiwc[:,i,1].min())

In [ ]:
bins = np.logspace(np.log10(0.8), np.log10(1), num=50)

fig, ax = plt.subplots()

histtype = 'bar'
ax.hist(spearman_tiwp[:,0], bins=bins, density=True, histtype=histtype, label='tiwp')
ax.hist(spearman_tiwp_fpavg[:,0], bins=bins, density=True, histtype=histtype, label='tiwp_fpavg')
ax.hist(spearman_tiwc[:,:,0].flatten(), bins=bins, density=True, histtype=histtype, label='tiwc')

ax.legend()
ax.minorticks_on()
ax.set_xscale('log')
ax.set_xlabel('$\\rho_S$')
ax.set_ylabel('PF')

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6/1.2, 4/1.2))

ax_tiwp = ax
ax_tiwc = ax.twiny()
ax_tiwc.invert_xaxis()

n = 1
alpha = 0.3
for label, array, ax_, color in zip(['tiwp', 'tiwc'], [spearman_tiwp, spearman_tiwc], [ax_tiwp, ax_tiwc], ['C0', 'C1']):
    idxs = np.argsort(array[...,0].flatten())[:n]
    quantile_levels = eval(f'ds.quantiles_{label[:4]}')
    ds_distribution = eval(f'ds.{label}_distribution').values.reshape(-1, quantile_levels.size)
    ds_distribution_isotonic = eval(f'ds.{label}_distribution_isotonic').values.reshape(-1, quantile_levels.size)

    for idx in idxs:
        qpd = ds_distribution[idx]
        qpd_corrected = ds_distribution_isotonic[idx]
        ax_.plot(qpd, quantile_levels, color=color)
        ax_.plot(qpd_corrected, quantile_levels, color='k', ls='dotted')
        ax_.scatter(qpd, quantile_levels, color=color, marker='x', alpha=alpha)
        ax_.scatter(qpd_corrected, quantile_levels, color='k', marker='x', alpha=alpha)
        mean_qpd = qq.posterior_mean(qpd, quantile_levels, quantile_axis=0)
        mean_cqpd = qq.posterior_mean(qpd_corrected, quantile_levels, quantile_axis=0)
        ax_.axvline(mean_qpd, color=color)
        ax_.axvline(mean_cqpd, color='k', ls='dotted')
        print(mean_qpd, mean_cqpd)


ax_tiwp.set_xlim(0, 0.0025)
ax_tiwc.set_xlim(0.0005, 0)
ax_tiwc.grid(visible=False)


ax.set_ylabel('CDF')
ax_tiwp.set_xlabel('TIWP [$\si{\kilo\gram\per\square\meter}$]')
ax_tiwc.set_xlabel('TIWC [$\si{\gram\per\cubic\meter}$]')

ax.set_ylim(0, 1)

legend_elements = [
    matplotlib.lines.Line2D([0], [0], color='k', label='QPD'),
    matplotlib.lines.Line2D([0], [0], color='k', ls='dotted', label='CQPD'),
    matplotlib.lines.Line2D([0], [0], lw=0, marker='x', markerfacecolor='k', markeredgecolor='k', label='QRNN quantile level'),
    #matplotlib.lines.Line2D([0], [0], lw=0, marker='|', markerfacecolor='k', markeredgecolor='k', label='$\mathbb{E}$[QPD] or $\mathbb{E}$[CQPD]', markersize=10),
    matplotlib.lines.Line2D([0], [0], lw=0, marker='s', markerfacecolor='C0', markeredgecolor='C0', label='TIWP'),#label='$\\text{\\texttt{tiwp}}$'),
    matplotlib.lines.Line2D([0], [0], lw=0, marker='s', markerfacecolor='C1', markeredgecolor='C1', label='TIWC'),#label='$\\text{\\texttt{tiwc}}$'),
    ]

ax_tiwc.legend(handles=legend_elements, loc='lower left', framealpha=0.5, handlelength=1)

fig.savefig('quantile_crossing_worst_cases.pdf', bbox_inches='tight')

plt.show()

In [ ]:
qq_tiwp = qq.posterior_quantiles(ds.tiwp_distribution.values, ds.quantiles_tiwp.values, np.array([0.05, 0.95]), quantile_axis=1)
print((qq_tiwp[:,0] < qq_tiwp[:,1]).all())
qq_tiwp_fpavg = qq.posterior_quantiles(ds.tiwp_fpavg_distribution.values, ds.quantiles_tiwp.values, np.array([0.05, 0.95]), quantile_axis=1)
print((qq_tiwp_fpavg[:,0] < qq_tiwp_fpavg[:,1]).all())
qq_tiwc = qq.posterior_quantiles(ds.tiwc_distribution.values, ds.quantiles_tiwc.values, np.array([0.05, 0.95]), quantile_axis=2)
print((qq_tiwc[:,:,0] < qq_tiwc[:,:,1]).all())

In [ ]:
def rpd(x, x_c):
    delta = x - x_c
    l1_mean = (abs(x) + abs(x_c)) / 2
    return np.divide(delta, l1_mean, out=np.zeros_like(delta), where=(np.isclose(delta, 0) == False))

In [ ]:
# Mean
tiwp_mean_qpd = qq.posterior_mean(ds.tiwp_distribution.values, ds.quantiles_tiwp.values, quantile_axis=1)
tiwp_mean_cqpd = qq.posterior_mean(ds.tiwp_distribution_isotonic.values, ds.quantiles_tiwp.values, quantile_axis=1)

d1_tiwp_mean = rpd(tiwp_mean_qpd, tiwp_mean_cqpd)

tiwp_fpavg_mean_qpd = qq.posterior_mean(ds.tiwp_fpavg_distribution.values, ds.quantiles_tiwp.values, quantile_axis=1)
tiwp_fpavg_mean_cqpd = qq.posterior_mean(ds.tiwp_fpavg_distribution_isotonic.values, ds.quantiles_tiwp.values, quantile_axis=1)

d1_tiwp_fpavg_mean = rpd(tiwp_fpavg_mean_qpd, tiwp_fpavg_mean_cqpd)

tiwc_mean_qpd = qq.posterior_mean(ds.tiwc_distribution.values, ds.quantiles_tiwc.values, quantile_axis=2)
tiwc_mean_cqpd = qq.posterior_mean(ds.tiwc_distribution_isotonic.values, ds.quantiles_tiwc.values, quantile_axis=2)

d1_tiwc_mean = rpd(tiwc_mean_qpd, tiwc_mean_cqpd)

# 90% CI
tiwp_90ci_qpd = qq.posterior_quantiles(ds.tiwp_distribution.values, ds.quantiles_tiwp.values, np.array([0.05, 0.90]), quantile_axis=1)
tiwp_90ci_cqpd = qq.posterior_quantiles(ds.tiwp_distribution_isotonic.values, ds.quantiles_tiwp.values, np.array([0.05, 0.90]), quantile_axis=1)

d1_tiwp_90ci = rpd(tiwp_90ci_qpd, tiwp_90ci_cqpd)

tiwp_fpavg_90ci_qpd = qq.posterior_quantiles(ds.tiwp_fpavg_distribution.values, ds.quantiles_tiwp.values, np.array([0.05, 0.90]), quantile_axis=1)
tiwp_fpavg_90ci_cqpd = qq.posterior_quantiles(ds.tiwp_fpavg_distribution_isotonic.values, ds.quantiles_tiwp.values, np.array([0.05, 0.90]), quantile_axis=1)

d1_tiwp_fpavg_90ci = rpd(tiwp_fpavg_90ci_qpd, tiwp_fpavg_90ci_cqpd)

tiwc_90ci_mean_qpd = qq.posterior_quantiles(ds.tiwc_distribution.values, ds.quantiles_tiwc.values, np.array([0.05, 0.90]), quantile_axis=2)
tiwc_90ci_mean_cqpd = qq.posterior_quantiles(ds.tiwc_distribution_isotonic.values, ds.quantiles_tiwc.values, np.array([0.05, 0.90]), quantile_axis=2)

d1_tiwc_90ci = rpd(tiwc_90ci_mean_qpd, tiwc_90ci_mean_cqpd)

# Probability larger than 1e-3 kg/m2
p_tiwp_qpd = qq.probability_larger_than(ds.tiwp_distribution.values, ds.quantiles_tiwp.values, 1e-3, 1)
p_tiwp_cqpd = qq.probability_larger_than(ds.tiwp_distribution_isotonic.values, ds.quantiles_tiwp.values, 1e-3, 1)

d1_p_tiwp = rpd(p_tiwp_qpd, p_tiwp_cqpd)

p_tiwp_fpavg_qpd = qq.probability_larger_than(ds.tiwp_fpavg_distribution.values, ds.quantiles_tiwp.values, 1e-3, 1)
p_tiwp_fpavg_cqpd = qq.probability_larger_than(ds.tiwp_fpavg_distribution_isotonic.values, ds.quantiles_tiwp.values, 1e-3, 1)

d1_p_tiwp_fpavg = rpd(p_tiwp_fpavg_qpd, p_tiwp_fpavg_cqpd)

In [ ]:
fig, axs = plt.subplots(nrows=4, ncols=3)

axs[0,0].hist(d1_tiwp_mean, bins=np.linspace(-np.max(abs(d1_tiwp_mean)), np.max(abs(d1_tiwp_mean)), 25))

axs[0,1].hist(d1_tiwp_fpavg_mean, bins=np.linspace(-np.max(abs(d1_tiwp_fpavg_mean)), np.max(abs(d1_tiwp_fpavg_mean)), 25))

axs[0,2].hist(d1_tiwc_mean.flatten(), bins=np.linspace(-np.max(abs(d1_tiwc_mean)), np.max(abs(d1_tiwc_mean)), 25))

axs[1,0].hist(d1_tiwp_90ci[...,0], bins=np.linspace(-np.max(abs(d1_tiwp_90ci[...,0])), np.max(abs(d1_tiwp_90ci[...,0])), 25))

axs[1,1].hist(d1_tiwp_fpavg_90ci[...,0], bins=np.linspace(-np.max(abs(d1_tiwp_fpavg_90ci[...,0])), np.max(abs(d1_tiwp_fpavg_90ci[...,0])), 25))

axs[1,2].hist(d1_tiwc_90ci[...,0].flatten(), bins=np.linspace(-np.max(abs(d1_tiwc_90ci[...,0])), np.max(abs(d1_tiwc_90ci[...,0])), 25))

axs[2,0].hist(d1_tiwp_90ci[...,1], bins=np.linspace(-np.max(abs(d1_tiwp_90ci[...,1])), np.max(abs(d1_tiwp_90ci[...,1])), 25))

axs[2,1].hist(d1_tiwp_fpavg_90ci[...,1], bins=np.linspace(-np.max(abs(d1_tiwp_fpavg_90ci[...,1])), np.max(abs(d1_tiwp_fpavg_90ci[...,1])), 25))

axs[2,2].hist(d1_tiwc_90ci[...,1].flatten(), bins=np.linspace(-np.max(abs(d1_tiwc_90ci[...,1])), np.max(abs(d1_tiwc_90ci[...,1])), 25))

axs[3,0].hist(d1_p_tiwp, bins=np.linspace(-np.max(abs(d1_p_tiwp)), np.max(abs(d1_p_tiwp)), 25))

axs[3,1].hist(d1_p_tiwp_fpavg, bins=np.linspace(-np.max(abs(d1_p_tiwp_fpavg)), np.max(abs(d1_p_tiwp_fpavg)), 25))

fig.tight_layout()
plt.show()

In [ ]:
d1_dict = {
    'd1_tiwp_mean': d1_tiwp_mean,
    'd1_tiwp_fpavg_mean': d1_tiwp_fpavg_mean,
    # 'd1_tiwc_mean.flatten()': d1_tiwc_mean.flatten(),
    'd1_tiwp_90ci[...,0]': d1_tiwp_90ci[...,0],
    'd1_tiwp_fpavg_90ci[...,0]': d1_tiwp_fpavg_90ci[...,0],
    'd1_tiwp_90ci[...,0]': d1_tiwp_90ci[...,0],
    'd1_tiwp_fpavg_90ci[...,0]': d1_tiwp_fpavg_90ci[...,0],
    # 'd1_tiwc_90ci[...,0].flatten()': d1_tiwc_90ci[...,0].flatten(),
    'd1_tiwp_90ci[...,1]': d1_tiwp_90ci[...,1],
    'd1_tiwp_fpavg_90ci[...,1]': d1_tiwp_fpavg_90ci[...,1],
    # 'd1_tiwc_90ci[...,1].flatten()': d1_tiwc_90ci[...,1].flatten(),
    'd1_p_tiwp': d1_p_tiwp,
    'd1_p_tiwp_fpavg': d1_p_tiwp_fpavg,
}

In [ ]:
d1_values = np.concatenate([v for v in d1_dict.values()])

In [ ]:
fig, ax = plt.subplots()

ax.hist(d1_values, bins=np.linspace(-2, 2, 200), density=True)

plt.show()

In [ ]:
for tau in [0.0005, 0.005, 0.025, 0.05, 0.25, 0.5, 0.75, 0.95, 0.975, 0.995, 0.9995]:
    print(f'{tau:0.4f}: {np.quantile(d1_values, tau):+0.4f}')